<a href="https://colab.research.google.com/github/jamessat/neural-networks-demo/blob/main/Nina_Recognizer_with_libs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Build a neural network that recognizes the word "Nina" when typed into the console.

## Data preparation

### Subtask:
Create a dataset of text inputs, including instances of the word 'Nina' and other words/phrases. Label the data accordingly.


**Reasoning**:
Create the text data and corresponding labels, then convert them into a pandas DataFrame as requested.



In [ ]:
import pandas as pd

texts = [
    "Hello Nina",
    "Is Nina here?",
    "Nina is great",
    "Hello John",
    "How are you?",
    "This is a test",
    "Where is Nina?",
    "Good morning",
    "Nina, could you help?",
    "Another sentence without Nina"
]

labels = [
    1, # "Hello Nina"
    1, # "Is Nina here?"
    1, # "Nina is great"
    0, # "Hello John"
    0, # "How are you?"
    0, # "This is a test"
    1, # "Where is Nina?"
    0, # "Good morning"
    1, # "Nina, could you help?"
    0  # "Another sentence without Nina"
]

text_series = pd.Series(texts)
label_series = pd.Series(labels)

df = pd.DataFrame({'text': text_series, 'label': label_series})

display(df)

,text,label
0,Hello Nina,1
1,Is Nina here?,1
2,Nina is great,1
3,Hello John,0
4,How are you?,0
5,This is a test,0
6,Where is Nina?,1
7,Good morning,0
8,"Nina, could you help?",1
9,Another sentence without Nina,0


## Text vectorization

### Subtask:
Convert the text data into numerical representations that can be fed into a neural network. This could involve techniques like one-hot encoding, TF-IDF, or word embeddings.


**Reasoning**:
Import necessary libraries, instantiate the tokenizer, fit it on the text data, convert text to sequences, pad the sequences, and store them in a new variable.



In [ ]:
# Create a vocabulary of unique words
vocabulary = set()
for text in texts:
    for word in text.lower().split():
        vocabulary.add(word)

word_to_index = {word: i for i, word in enumerate(vocabulary)}

# Convert text data to sequences of indices
indexed_texts = []
for text in texts:
    indexed_texts.append([word_to_index[word] for word in text.lower().split() if word in word_to_index])

# Pad sequences to a fixed length (optional but often necessary for neural networks)
max_length = max(len(seq) for seq in indexed_texts)
padded_indexed_texts = []
for seq in indexed_texts:
    padded_seq = seq + [0] * (max_length - len(seq)) # Pad with 0s
    padded_indexed_texts.append(padded_seq)

display(padded_indexed_texts)

[[11, 8, 0, 0],
 [0, 8, 18, 0],
 [8, 0, 6, 0],
 [11, 13, 0, 0],
 [4, 3, 21, 0],
 [17, 0, 1, 22],
 [5, 0, 16, 0],
 [9, 19, 0, 0],
 [20, 10, 12, 14],
 [15, 2, 7, 8]]

## Model Building

### Subtask:
Design and build a neural network architecture suitable for text classification. This might involve layers like Embedding, LSTM or GRU, and Dense layers.

**Reasoning**:
Import the necessary PyTorch libraries, define a simple neural network class with an embedding layer, and linear layers for classification.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class NinaRecognizer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc1 = nn.Linear(embedding_dim * max_length, hidden_dim) # Linear layer after flattening the embedding output
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text):
        embedded = self.embedding(text)
        # Flatten the embedded output
        embedded = embedded.view(embedded.size(0), -1)
        hidden = self.fc1(embedded)
        activated = self.relu(hidden)
        output = self.fc2(activated)
        return self.sigmoid(output)

# Model hyperparameters
vocab_size = len(vocabulary)
embedding_dim = 16
hidden_dim = 32
output_dim = 1
pad_idx = 0 # Assuming 0 is the padding index

# Instantiate the model
model = NinaRecognizer(vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx)

# Define loss function and optimizer
criterion = nn.BCELoss() # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters())

print(model)

NinaRecognizer(
  (embedding): Embedding(23, 16, padding_idx=0)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


## Model Training

### Subtask:
Train the neural network model on the prepared and vectorized data.

**Reasoning**:
Convert the data to PyTorch tensors, create a TensorDataset and DataLoader, and implement the training loop.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors
# Convert padded_indexed_texts to a tensor of type Long
X_tensor = torch.LongTensor(padded_indexed_texts)
# Convert labels to a tensor of type Float
y_tensor = torch.FloatTensor(labels).unsqueeze(1) # Add a dimension for BCELoss

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Create a DataLoader
batch_size = 4 # You can adjust the batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
num_epochs = 100 # You can adjust the number of epochs

for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished.")

Epoch [10/100], Loss: 0.5421
Epoch [20/100], Loss: 0.3788
Epoch [30/100], Loss: 0.1809
Epoch [40/100], Loss: 0.1678
Epoch [50/100], Loss: 0.1596
Epoch [60/100], Loss: 0.0488
Epoch [70/100], Loss: 0.0520
Epoch [80/100], Loss: 0.0354
Epoch [90/100], Loss: 0.0044
Epoch [100/100], Loss: 0.0103
Training finished.


## Model Evaluation

### Subtask:
Evaluate the trained model's performance using appropriate metrics.

**Reasoning**:
Use the trained model to make predictions on the training data and compare them to the actual labels.

In [ ]:
# Set the model to evaluation mode
model.eval()

# Make predictions on the training data
with torch.no_grad(): # Disable gradient calculation for evaluation
    predictions = model(X_tensor)
    # Convert predictions to binary (0 or 1) based on a threshold (e.g., 0.5)
    predicted_labels = (predictions > 0.5).int()

# Compare predicted labels to actual labels
accuracy = (predicted_labels == y_tensor).float().mean()

print(f"Model Accuracy on Training Data: {accuracy.item():.4f}")

# Display predictions and actual labels for review
# Add predicted labels as a new column to the existing DataFrame
df['Predicted Label'] = predicted_labels.squeeze().tolist()
display(df)

Model Accuracy on Training Data: 1.0000


,text,label,Predicted Label
0,Hello Nina,1,1
1,Is Nina here?,1,1
2,Nina is great,1,1
3,Hello John,0,0
4,How are you?,0,0
5,This is a test,0,0
6,Where is Nina?,1,1
7,Good morning,0,0
8,"Nina, could you help?",1,1
9,Another sentence without Nina,0,0


## Prediction Function

### Subtask:
Create a function that takes a text input from the console, preprocesses it (vectorization), and uses the trained model to predict if the word 'Nina' is present.

**Reasoning**:
Define a function that takes a text string as input, preprocesses it using the vocabulary and max length from data preparation, converts the processed input to a PyTorch tensor, makes a prediction using the trained model, and returns the result.

In [ ]:
def predict_nina(text, model, word_to_index, max_length, pad_idx):
    model.eval() # Set the model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation

        # Preprocess the input text
        indexed_text = [word_to_index.get(word, pad_idx) for word in text.lower().split()]
        # Pad the indexed text
        padded_indexed_text = indexed_text + [pad_idx] * (max_length - len(indexed_text))
        # Convert to PyTorch tensor and add a batch dimension
        input_tensor = torch.LongTensor([padded_indexed_text])

        # Get prediction from the model
        output = model(input_tensor)
        # Apply threshold to get binary prediction
        prediction = (output > 0.5).item() # .item() to get a scalar value

        return prediction

# Example usage of the prediction function (optional)
# print(predict_nina("Hello Nina", model, word_to_index, max_length, pad_idx))
# print(predict_nina("Hello John", model, word_to_index, max_length, pad_idx))

## Console Interface

### Subtask:
Implement a loop to continuously read input from the console and use the prediction function to determine if 'Nina' was typed.

**Reasoning**:
Create a loop that prompts the user for input, calls the `predict_nina` function with the input, and prints the prediction result. Include a condition to break the loop when the user types 'quit'.

In [ ]:
print("Nina Recognizer - Type 'quit' to exit")

while True:
    user_input = input("Enter text: ")
    if user_input.lower() == 'quit':
        break

    prediction = predict_nina(user_input, model, word_to_index, max_length, pad_idx)

    if prediction == 1:
        print("Recognition: 'Nina' is recognized.")
    else:
        print("Recognition: 'Nina' is not recognized.")

print("Exiting Nina Recognizer.")

Nina Recognizer - Type 'quit' to exit
Enter text: james
Recognition: 'Nina' is recognized.
Enter text: makr
Recognition: 'Nina' is recognized.
Enter text: Nina
Recognition: 'Nina' is recognized.
Enter text: Tom
Recognition: 'Nina' is recognized.
Enter text: A
Recognition: 'Nina' is not recognized.
Enter text: B
Recognition: 'Nina' is recognized.
Enter text: N
Recognition: 'Nina' is recognized.
Enter text: C
Recognition: 'Nina' is recognized.
Enter text: D
Recognition: 'Nina' is recognized.
Enter text: E
Recognition: 'Nina' is recognized.
Enter text: quit
Exiting Nina Recognizer.
